# Title

Project website: https://docs.google.com/document/d/1GkK28wOyjTPZEZuOumKPU0z1NzVT_9sxETjPOcLPVYo/edit?tab=t.0#heading=h.qifoo7co6qtd

Professor website: https://malchiodi.di.unimi.it/teaching/AMD-DSE/2024-25/en

In [ ]:
# Install required packages from the requirement.txt file if not already installed
!pip install -r requirements.txt

In [ ]:
import subprocess
import sys
import os
import pandas as pd

# Set environment variables for Kaggle API
os.environ['KAGGLE_USERNAME'] = "ilchurch"
os.environ['KAGGLE_KEY'] = "5449548f4b6c8556c8f48f4a31b4ea6e"

# Ensure kaggle is installed
try:
    import kaggle
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "kaggle"])
    import kaggle

# Import and authenticate using KaggleApi
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

# Download and unzip the dataset
api.dataset_download_files('mohamedbakhet/amazon-books-reviews', path='data', unzip=True)

In [7]:
# amazon_books_data
# The dataset contains the following files:
# - books_data.csv: 
# - Books_rating.csv:

In [8]:
# Load the dataset with spark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("JaccardSimilarity") \
    .master("local[*]") \
    .getOrCreate()
books_rating = spark.read.csv("data/Books_rating.csv", header=True, inferSchema=True)

ModuleNotFoundError: No module named 'pyspark.conf'

In [ ]:
# Display the first few rows of the books_rating dataset
books_rating.show(5)

# Create a subsample of the dataset for performance reasons
I will use a sample of the dataset as a test case. My local and Google Collab computers are not powerful enough to handle the full dataset. I will use *sample* as a parameter can it can be changed to the desired sample size.

In [ ]:
sample = 0.01 # 1% of the dataset

In [ ]:
books_rating_sample = books_rating.sample(fraction=sample, seed=42)
books_rating_sample.show(5)
books_rating_sample.count()

# Checking for null values in the 'review/text' column
Since we are using text data, we need to check for null values in the 'review/text' column in order to take a decision on how to handle them.

In [ ]:
# Check for null values in the 'review/text' column
null_rows_df = books_rating.filter(
    books_rating['review/text'].isNull() | (books_rating['review/text'].rlike('^\\s*$'))
)

null_rows_df.show(30)

Since the 'review/text' null values are more or less duplicates. 

They share the same columns, besides the identifiers and the Title. Although the Title is not the same, with a qualitative check we can see they are about the same book.

In [ ]:
from pyspark.sql.functions import col
filtered_df = books_rating.filter(
    col("Title").contains("Lord of the Rings")
).select("Title", "review/text").limit(100)

# Show the result
filtered_df.show(100, truncate=False)

There are many others Lord of The Rings reviews. I think we can't discard the NA values and it's better to keep them. We can use also the Title as similarity check.

In [ ]:
# Check for null values in the 'review/text' column
null_rows_df_sample = books_rating_sample.filter(
    books_rating_sample['review/text'].isNull() | (books_rating_sample['review/text'].rlike('^\\s*$'))
)

null_rows_df_sample.show(30)

The is a null value in the subsample

# Working with the subsample

## Jaccard Similarity Approach
Since the dataset is too big, I will use a subsample of the dataset to test the Jaccard Similarity approach, for test purposes. I will then try top scale it to the full dataset.
In addition, I will use the MapReduce approach to parallelize the computation of the Jaccard Similarity.

In [ ]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover
from pyspark.sql.functions import col, udf
from pyspark.sql.types import ArrayType, StringType

In [ ]:
tokenizer = RegexTokenizer(inputCol="review/text", outputCol="reviews/tokens", pattern="\\W") # \\W is a regex pattern that matches any non-word character

It tokenizes the text into words. I am trying this approach because I believe it will be faster then some neural network based approach, since this use sparks. It will have some limitations, since it does not take in account of compound words, but I think it will be good for the first approach.

In [ ]:
stopwords_remover = StopWordsRemover(inputCol="reviews/tokens", outputCol="tokens/nostopwords")

StopWordsRemover() uses a predefined list of stop words in English. It removes common words that do not carry much meaning, such as "the", "is", "in", etc. This is important, since the Jaccard Similarity is based on the number of common words between two texts. If we don't remove the stop words, the Jaccard Similarity will be biased by the number of stop words in the texts, which are not relevant for the meaning of the texts.

In [ ]:
def remove_numbers(tokens):
    return [token for token in tokens if not token.isdigit()]

Numbers are often not relevant for the meaning of a review. In our dataset, the score is stored in a separated column, so we can remove the numbers from the text.

In [ ]:
remove_numbers_udf = udf(remove_numbers, ArrayType(StringType())) # Specify the return type as ArrayType(StringType), meaning an array/list of strings

Sparks does not work as a Python object, so we need UDF (user defined function) to use it in the pipeline.

In [ ]:
tokenized_books_rating_sample = tokenizer.transform(books_rating_sample) # Apply the tokenizer to the dataset
tokenized_reviews_nostopwords_sample = stopwords_remover.transform(tokenized_books_rating_sample) # Remove the stop words from the tokenized dataset
tokenized_rw_nsw_nn_sample = tokenized_reviews_nostopwords_sample.withColumn("tokens/nostopwords/nonumbers", remove_numbers_udf(col("tokens/nostopwords"))) # Remove the numbers from the dataset

In [ ]:
tokenized_rw_nsw_nn_sample.show(10)